In [1]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 43kB/s 
     |████████████████████████████████| 3.8MB 38.5MB/s 
     |████████████████████████████████| 450kB 44.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4f5934f1940859018567511599006f08bafae4f9636f42a61a33045dc7c794ae
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstal

In [2]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fid = drive.ListFile({'q':"title='asl_data.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('asl_data.zip')

In [0]:
import os

In [7]:
os.listdir()

['.config', 'adc.json', 'asl_data.zip', 'sample_data']

In [8]:
!unzip asl_data.zip

Archive:  asl_data.zip
   creating: asl_data/
  inflating: asl_data/asl_data.p     
  inflating: asl_data/asl_target.p   


In [0]:
import pickle

In [0]:
x=pickle.load(open("asl_data/asl_data.p","rb"))

In [12]:
x

array([[[[255,   8,   0],
         [249,   4,   0],
         [245,   3,   0],
         ...,
         [250,   0,  10],
         [246,   0,   4],
         [252,   2,   7]],

        [[252,   3,   0],
         [238,   7,   0],
         [203,   7,   3],
         ...,
         [191,   0,  15],
         [214,   4,  14],
         [216,   0,   5]],

        [[247,   0,   7],
         [208,   6,  13],
         [220, 110, 112],
         ...,
         [153,  84, 104],
         [206,  78,  95],
         [165,  10,  25]],

        ...,

        [[248,   1,   3],
         [197,   7,   8],
         [105,  25,  26],
         ...,
         [148, 136, 148],
         [218, 129, 139],
         [137,  10,  19]],

        [[249,   2,   0],
         [214,   5,   1],
         [152,  17,  15],
         ...,
         [215, 126, 135],
         [254, 118, 122],
         [171,  12,  14]],

        [[252,   4,   0],
         [223,   5,   0],
         [176,  13,  10],
         ...,
         [140,  14,  19],
        

In [0]:
import numpy as np
xi=np.divide(x,255.0)

In [0]:
y=pickle.load(open("asl_data/asl_target.p","rb"))
y=np.asarray(y)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout

In [0]:
model=Sequential([
                  Conv2D(64,(3,3),activation='relu',input_shape=(200,200,3)),
                  MaxPooling2D(),
                  Dropout(0.2),
                  Conv2D(32,(3,3),activation='relu'),
                  MaxPooling2D(),
                  Conv2D(64,(3,3),activation='relu'),
                  MaxPooling2D(),
                  Dropout(0.2),
                  Flatten(),
                  Dense(512,activation='relu'),
                  Dense(29,activation='softmax')



])

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [23]:
history=model.fit(x,y,batch_size=100,
          epochs=7,
          validation_split=0.1)

Train on 2610 samples, validate on 290 samples
Epoch 1/7
2610/2610 [==============================] - 248s 95ms/sample - loss: 72.9717 - accuracy: 0.0678 - val_loss: 3.2591 - val_accuracy: 0.1241
Epoch 2/7
2610/2610 [==============================] - 243s 93ms/sample - loss: 2.6836 - accuracy: 0.2613 - val_loss: 3.0446 - val_accuracy: 0.2172
Epoch 3/7
2610/2610 [==============================] - 243s 93ms/sample - loss: 2.0162 - accuracy: 0.4479 - val_loss: 2.9833 - val_accuracy: 0.3207
Epoch 4/7
2610/2610 [==============================] - 244s 94ms/sample - loss: 1.4032 - accuracy: 0.6023 - val_loss: 2.9481 - val_accuracy: 0.3448
Epoch 5/7
2610/2610 [==============================] - 245s 94ms/sample - loss: 1.0856 - accuracy: 0.6893 - val_loss: 2.6149 - val_accuracy: 0.4414
Epoch 6/7
2610/2610 [==============================] - 244s 93ms/sample - loss: 0.9777 - accuracy: 0.7341 - val_loss: 3.4329 - val_accuracy: 0.4034
Epoch 7/7
2610/2610 [==============================] - 243s 93ms

In [0]:
import plotly.graph_objects as go


In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                        y=history.history['accuracy'],
                        mode='lines+markers',
                        name='Training accuraccy'))
fig.add_trace(go.Scatter(
    x=history.epoch,
    y=history.history['val_accuracy'],
    mode='lines+markers',
    name='validation accuracy'
    
))

fig.update_layout(
    title='Accuracy',
    xaxis=dict(title='Epoch'),
    yaxis=dict(title='Percentage')
)

fig.show()


In [27]:
fig.write_image("fig.png")

ValueError: ignored

In [28]:
model.save("model.m")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model.m/assets


In [29]:
os.listdir()

['.config', 'model.m', 'adc.json', 'asl_data.zip', 'asl_data', 'sample_data']

In [30]:
os.listdir(os.path.join(os.getcwd,'adc.json'))

TypeError: ignored

In [32]:
path=os.path.join(os.getcwd,'asl_data')

TypeError: ignored

In [0]:
file=drive.CreateFile({'title':"model.m"})

In [36]:
os.listdir()

['.config', 'model.m', 'adc.json', 'asl_data.zip', 'asl_data', 'sample_data']

In [0]:
path=os.path.join(os.getcwd(),'asl_data')

In [0]:
import os

In [39]:
os.listdir()

['.config', 'model.m', 'adc.json', 'asl_data.zip', 'asl_data', 'sample_data']

In [0]:
from tensorflow import keras


In [0]:
model=keras.models.load_model("model.m")

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 198, 198, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 99, 99, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 97, 97, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 46, 46, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 64)       

In [43]:
ddmodel.accuracy()

AttributeError: ignored

In [44]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [46]:
!ls /content/gdrive/'My Drive'


 asl  'Colab Notebooks'   data_asl  'Getting started.pdf'


In [0]:
import torch

In [0]:
model_save_name = 'asl_model.m'
path = "/content/gdrive/My Drive/{k}".format(k=model_save_name)


In [52]:
model.save(path)

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/asl_model.m/assets


In [0]:
model.save("/content/gdrive/My Drive/asl_model.h5")